In [1]:
import ee
ee.Initialize()

In [2]:
from ipygee import *

## Create a Map instance
- Arguments:
  - tabs: a `tuple` indicating which tabs to load in the map. Options are: Inspector, Layers, Assets, Tasks
  - kwargs: as this class inherits from `ipyleaflet.Map` it can accept all its arguments

In [3]:
Map = Map()

## Show map with method `show`
- Arguments
  - tabs: show tabs (bool)
  - layer_control: show a control for layers (bool)
  - draw_control: show a control for drawings (bool)
  - fullscrean: show fullscreen button (bool)

In [4]:
Map.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map data (c) <a href…

### Resize Map
Dimensions must be in pixel units

In [5]:
Map.setDimensions('70%', '500px')

## Load some Earth Engine Objects

In [6]:
site = ee.Geometry.Point([-72, -42])
bounds = site.buffer(10000)

collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(bounds).filterDate('2019-01-01', '2019-02-01')
image = collection.first()

countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
sa = countries.filterMetadata('wld_rgn', 'equals', 'South America')
arg = ee.Feature(countries.filterMetadata('country_na', 'equals', 'Argentina').first())

## Define visualization parameters

In [7]:
visParam = {'bands':['B8', 'B11', 'B4'], 'min':0, 'max':5000}

## Add Layers

In [8]:
Map.addLayer(site, name='site point')
Map.addLayer(bounds, name='site bounds')
Map.addLayer(image, visParam, name='Sentinel 2 Image')

## Add all images from a collection

In [9]:
Map.addImageCollection(collection.limit(5), visParam, 
                       namePattern='S2 {system_date} clouds: {CLOUD_COVERAGE_ASSESSMENT}%')

## Add FeatureCollection and Feature
- visualization parameters:
  - outline_color: color for the outline
  - outline:  width of the outline
  - fill_color: color for the inside

In [10]:
Map.addLayer(sa, {'outline_color':'red'}, name='South America')
Map.addLayer(arg, {'fill_color':'blue', 'outline': 1}, name='Argentina')

# Center object

In [11]:
Map.centerObject(bounds)

## Get attributes

In [12]:
# Zoom
Map.zoom

11

In [13]:
# ipyleaflet bounds (not Earth Engine)
Map.bounds

((-65.6582745198266, -118.12500000000001),
 (65.65827451982663, 118.12500000000001))

In [14]:
# Earth Engine Bounds
Map.getBounds().getInfo()

{'coordinates': [[[-118.12500000000001, -65.6582745198266],
   [-118.12500000000001, 65.65827451982663],
   [118.12500000000001, 65.65827451982663],
   [118.12500000000001, -65.6582745198266],
   [-118.12500000000001, -65.6582745198266]]],
 'type': 'Polygon'}

In [15]:
# ipyleaflet center (not Earth Engine)
Map.center

[-42.00000308012648, -71.9998286857986]

In [16]:
# Earth Engine Center
Map.getCenter().getInfo()

{'coordinates': [-71.9998286857986, -42.00000308012648], 'type': 'Point'}

## Layers
All Layers are contained in an `OrderedDict` where keys are the name of the layers and the value is a `dict` with the following keys:
- layer: `ipyleaflet.TileLayer` added to the Map
- object: Earth Engine Object used to create the Layer
- type: type of EE Object
- visParams: visualization parametes

All layers can be accesed with `Map.EELayers`

### Get one layer by its name

In [17]:
layer = Map.getLayer('Sentinel 2 Image')

In [18]:
layer

{'layer': TileLayer(attribution='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ', name='Sentinel 2 Image', options=['min_zoom', 'detect_retina', 'attribution', 'max_zoom', 'tile_size'], url='https://earthengine.googleapis.com/map/12d55689141a1230e95c5f1668144339/{z}/{x}/{y}?token=af553ad8bdd65700c3d7bd80297a96b7'),
 'object': <ee.image.Image at 0x7fbfb814ec88>,
 'type': 'Image',
 'visParams': {'bands': 'B8,B11,B4', 'max': 5000, 'min': 0}}

### Directly get the asociated EE object

In [19]:
eelayer = Map.getObject('Sentinel 2 Image')

In [20]:
eelayer.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B10',
 'B11',
 'B12',
 'QA10',
 'QA20',
 'QA60']

# TABS
## You can add a custom Tab with a custom handler. The handler is a function with 4 main parameters:
- **type:** the interaction type. Can be 'click', 'mouseover', etc
- **coordinates:** the coordinates where the interaction has taken place. If you have used ipyleaflet before, take in consideraton that coordinates are inverted (to match GEE format): [longitud, latitude]
- **widget:** The widget inside the Tab. Defaults to an empty HTML widget
- **map:** the Map instance. You can apply any of its methods, or get any of its properties

In [21]:
print(Map.addTab.__doc__)

 Add a Tab to the Panel. The handler is for the Map

        :param name: name for the new tab
        :type name: str
        :param handler: handle function for the new tab. Arguments of the
        function are:

          - type: the type of the event (click, mouseover, etc..)
          - coordinates: coordinates where the event occurred [lon, lat]
          - widget: the widget inside the Tab
          - map: the Map instance

        :param widget: widget inside the Tab. Defaults to HTML('')
        :type widget: ipywidgets.Widget
        


In [22]:
def test_handler(**change):    
    # PARAMS
    ty = change['type']
    coords = change['coordinates']
    wid = change['widget']
    themap = change['map']
        
    if ty == 'click':  # If interaction was a click
        # Loading message before sending a request to EE
        wid.value = 'Loading...'
        # Map's bounds
        bounds = themap.getBounds().getInfo()['coordinates']
        # Change Widget Value
        wid.value = "You have clicked on {} and map's bounds are {}".format(coords, bounds)        

Map.addTab('TestTAB', test_handler)
print("Check out the Map!")

Check out the Map!
